In [1]:
data = 'final_data.csv'

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import operator
import string
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
#Import necessary libraries
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Utility
import os
from collections import Counter

In [ ]:
df = pd.read_csv(data)

In [ ]:
df.isnull().sum()

In [ ]:
#drop null values
df.dropna(inplace=True)

In [ ]:
df.rename(columns={'Text':'text','Mental_condition':'label'},inplace=True)
df['length'] = df['text'].apply(len)

In [ ]:
df.head()

In [ ]:
df.label.value_counts().plot(kind='bar')

In [ ]:
'''
different preprocess functions
'''
def preprocess(Text):
    Text = Text.str.replace("(<br/>)", "")
    Text = Text.str.replace('(<a).*(>).*(</a>)', '')
    Text = Text.str.replace('(&amp)', '')
    Text = Text.str.replace('(&gt)', '')
    Text = Text.str.replace('(&lt)', '')
    Text = Text.str.replace('(\xa0…)', ' ')  
    Text = Text.str.replace('(\n)', ' ')
    Text = Text.str.replace('+', '')
    return Text

# remove urls from text
def remove_urls(row):
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    row =  url_pattern.sub(r'', row)
    return row

# remove @accounts (@[A-Za-z0–9]+)|, emojis，#hashtags, com frin text
def removes(row):
    url_pattern = re.compile(r"(\w+:\/\/\S+)|(#(\w+))|(.twitter.com/[A-Za-z0–9]+)")
    row =  url_pattern.sub(r'', row)
    return row

def removes(row):
    url_pattern = re.compile(r"(\w+:\/\/\S+)|(#(\w+))|(.twitter.com/[A-Za-z0–9]+)")
    row =  url_pattern.sub(r'', row)
    return row
def decontracted(phrase):
    phrase = re.sub(r"that\'s", "that is", phrase)   
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#apply those functions
df['text'] = preprocess(df.text)
df['text'] = df['text'].apply(remove_urls)
df['text'] = df['text'].apply(removes)
df.text = df.text.apply(lambda x: decontracted(x).lower())


In [ ]:
stop_words = stopwords.words('english')
not_stopwords = {}
common_words =['com','twitter','please',
               'the','would','could','can','may','must', 
              # 'us','i','you','this','we','me','that','it',
               'de','s','pic','t','m','en','la','que','don','re','e','el','ca']
stop_words.extend(word for word in common_words if word not in stop_words)
final_stop_words = set([word for word in stop_words if word not in not_stopwords])
df.text = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in final_stop_words]))
rm_punc = re.compile(r"(\W)")
rm_space = re.compile(r"(\s+)")
df.text = df.text.apply(lambda x: rm_punc.sub(' ', x))
df.text = df.text.apply(lambda x: rm_space.sub(' ', x))

In [ ]:
classes = list(df.label.unique())
text = list(df.text.values)
X = [create_vector(i) for i in text]
Y = list(df.label.values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfTransformer as TF
from sklearn.model_selection import train_test_split as tts
from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.pipeline import Pipeline
import string

In [ ]:
X_train, X_test, y_train, y_test = tts(df.text, df.label, random_state = 42, test_size = 0.15)

In [ ]:
nb = Pipeline([('vect', CV()),
               ('clf', MB()),
              ])

In [ ]:
nb.fit(X_train,y_train)

In [ ]:
labels = ['pos','dep','sucide']

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=labels))